# Розширення генерації на основі пошуку (RAG) та векторні бази даних

In [ ]:
!pip install getenv openai==1.12.0 faiss-cpu pandas numpy

In [7]:
import os
import pandas as pd
import numpy as np
import openai
import faiss

## Створення нашої бази знань

Налаштування FAISS для векторного пошуку


In [ ]:
# Перевірка встановлення FAISS
!pip install faiss-cpu --no-cache-dir

In [4]:
# Ініціалізація індексу FAISS
# Буде створено пізніше, коли знатимемо розмірність векторів
# Типовий розмір ембедінгів ada-002 - 1536 вимірювань

In [ ]:
import pandas as pd

# Ініціалізація порожнього DataFrame
df = pd.DataFrame(columns=['path', 'text'])


# розбиття наших даних на фрагменти
data_paths= ["data/frameworks.md?WT.mc_id=academic-105485-koreyst", "data/own_framework.md?WT.mc_id=academic-105485-koreyst", "data/perceptron.md?WT.mc_id=academic-105485-koreyst"]

for path in data_paths:
    with open(path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Додаємо шлях до файлу та текст до DataFrame
    df = df.append({'path': path, 'text': file_content}, ignore_index=True)

df.head()

In [10]:
def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    # Якщо останній фрагмент не досягнув мінімальної довжини, все одно додати його
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Припускаючи, що analyzed_df - це pandas DataFrame, а 'output_content' - це стовпець у цьому DataFrame
splitted_df = df.copy()
splitted_df['chunks'] = splitted_df['text'].apply(lambda x: split_text(x, 400, 300))

splitted_df

In [11]:
# Припускаючи, що 'chunks' - це стовпець списків у DataFrame splitted_df, ми розділимо фрагменти на різні рядки
flattened_df = splitted_df.explode('chunks')

flattened_df.head()

## Перетворення нашого тексту на ембедінги

Перетворення нашого тексту на ембедінги та зберігання їх для використання з FAISS

In [12]:
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY") 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-07-01-preview"



In [13]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"))

In [14]:
def create_embeddings(text, model="text-embedding-ada-002-2"):
    # Створити ембедінги для кожного фрагмента документа
    embeddings = openai.embeddings.create(input = text, model=model).data[0].embedding
    return embeddings

#ембедінги для першого фрагмента
create_embeddings(flattened_df['chunks'][0])

In [15]:
cat = create_embeddings("cat")
cat

In [16]:
# створити ембедінги для всіх фрагментів даних і зберегти їх у списку

embeddings = []
for chunk in flattened_df['chunks']:
    embeddings.append(create_embeddings(chunk))

# зберегти ембедінги в DataFrame
flattened_df['embeddings'] = embeddings

flattened_df.head()

# Пошук з використанням FAISS

Векторний пошук та схожість між нашим запитом і базою даних з використанням FAISS

### Створення індексу FAISS та підготовка до пошуку

In [17]:
import numpy as np

# Отримуємо ембедінги як масив numpy
embeddings_list = flattened_df['embeddings'].to_list()
embeddings_array = np.array(embeddings_list).astype('float32')

# Визначаємо розмірність векторів (для ембедінгів ada-002 це 1536)
vector_dimension = len(embeddings_list[0])

# Створюємо індекс FAISS
index = faiss.IndexFlatL2(vector_dimension)  # L2 - це евклідова відстань

# Додаємо наші вектори до індексу
index.add(embeddings_array)

# Перевіряємо кількість векторів в індексі
print(f"Загальна кількість векторів в індексі: {index.ntotal}")

In [18]:
# Ваше текстове запитання
question = "what is a perceptron?"

# Перетворіть запитання у вектор запиту
query_vector = create_embeddings(question)  
query_vector_array = np.array([query_vector]).astype('float32')

# Знайдіть найбільш схожі документи (k=5 - скільки найближчих сусідів шукаємо)
k = 5
distances, indices = index.search(query_vector_array, k)

# Виведіть найбільш схожі документи
for i in range(min(3, len(indices[0]))):
    idx = indices[0][i]
    print(f"Фрагмент {i+1}:")
    print(flattened_df['chunks'].iloc[idx])
    print(f"Шлях: {flattened_df['path'].iloc[idx]}")
    print(f"Відстань: {distances[0][i]}")
    print("-" * 50)

## Поєднання всього для відповіді на запитання

In [22]:
import os
import openai

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

In [24]:
user_input = "what is a perceptron?"

def chatbot(user_input):
    # Перетворіть запитання у вектор запиту
    query_vector = create_embeddings(user_input)
    query_vector_array = np.array([query_vector]).astype('float32')
    
    # Знайдіть найбільш схожі документи з FAISS
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)

    # додайте документи до запиту, щоб забезпечити контекст
    history = []
    for idx in indices[0]:
        history.append(flattened_df['chunks'].iloc[idx])

    # поєднайте історію та введення користувача
    history.append(user_input)

    # створіть об'єкт повідомлення
    messages=[
        {"role": "system", "content": "You are an AI assiatant that helps with AI questions."},
        {"role": "user", "content": history[-1]}
    ]

    # використайте завершення чату для генерації відповіді
    response = openai.chat.completions.create(
        model="gpt-35-turbo-1106",
        temperature=0.7,
        max_tokens=800,
        messages=messages
    )

    return response.choices[0].message

chatbot(user_input)

## Тестування та оцінка

Базовий приклад того, як можна використовувати середню точність (Mean Average Precision, MAP) для оцінки відповідей вашої моделі на основі їх релевантності.

In [25]:
from sklearn.metrics import average_precision_score

# Визначте ваші тестові випадки
test_cases = [
    {
        "query": "What is a perceptron?",
        "relevant_responses": ["A perceptron is a type of artificial neuron.", "It's a binary classifier used in machine learning."],
        "irrelevant_responses": ["A perceptron is a type of fruit.", "It's a type of car."]
    },
    {
        "query": "What is machine learning?",
        "relevant_responses": ["Machine learning is a method of data analysis that automates analytical model building.", "It's a branch of artificial intelligence based on the idea that systems can learn from data, identify patterns and make decisions with minimal human intervention."],
        "irrelevant_responses": ["Machine learning is a type of fruit.", "It's a type of car."]
    },
    {
        "query": "What is deep learning?",
        "relevant_responses": ["Deep learning is a subset of machine learning in artificial intelligence (AI) that has networks capable of learning unsupervised from data that is unstructured or unlabeled.", "It's a type of machine learning."],
        "irrelevant_responses": ["Deep learning is a type of fruit.", "It's a type of car."]
    },
    {
        "query": "What is a neural network?",
        "relevant_responses": ["A neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates.", "It's a type of machine learning."],
        "irrelevant_responses": ["A neural network is a type of fruit.", "It's a type of car."]
    }
]

# Ініціалізуйте загальну середню точність
total_average_precision = 0

# Тестування програми RAG
for test_case in test_cases:
    query = test_case["query"]
    relevant_responses = test_case["relevant_responses"]
    irrelevant_responses = test_case["irrelevant_responses"]

    # Генерування відповіді за допомогою вашої програми RAG
    response = chatbot(query) 

    # Створіть список усіх відповідей та список істинних бінарних міток
    all_responses = relevant_responses + irrelevant_responses
    true_labels = [1] * len(relevant_responses) + [0] * len(irrelevant_responses)

    # Створіть список прогнозованих оцінок на основі того, чи є відповідь згенерованою відповіддю
    predicted_scores = [1 if resp == response else 0 for resp in all_responses]

    # Розрахуйте середню точність для цього запиту
    average_precision = average_precision_score(true_labels, predicted_scores)

    # Додайте середню точність до загальної середньої точності
    total_average_precision += average_precision

# Обчисліть середню точність
mean_average_precision = total_average_precision / len(test_cases)

In [26]:
mean_average_precision